In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("data/opta_sample_data_2.csv")

In [8]:
df.head()

,ID,GAME_ID,TEAM_ID,TEAM_NAME,GOALS,FIRST_HALF_GOALS,POSSESSION_PERCENTAGE,TOTAL_PASS,EVENT_TYPE_ID,EVENT_NAME,EVENT_PERIOD_MINUTE,HOME_TEAM,GAME_DATE
0,1848890342,919266,14,Liverpool,4,2,72.3,737,1,Pass,76,14,2018-05-13T15:00:00Z
1,1119073579,919266,36,Brighton and Hove Albion,0,0,27.7,275,1,Pass,47,14,2018-05-13T15:00:00Z
2,1848890342,919266,14,Liverpool,4,2,72.3,737,1,Pass,16,14,2018-05-13T15:00:00Z
3,1848890342,919266,14,Liverpool,4,2,72.3,737,1,Pass,2,14,2018-05-13T15:00:00Z
4,1119073579,919266,36,Brighton and Hove Albion,0,0,27.7,275,10,Save,32,14,2018-05-13T15:00:00Z


# Create complete event prompt

In [9]:
df_goals = df.query("EVENT_NAME == 'Goal'")
goal_events = {
    'Team': df_goals.TEAM_NAME.values.tolist(),
    'Event period minute': df_goals.EVENT_PERIOD_MINUTE.values.tolist()
}

event_prompt = [str(team) + " scoorde in minuut " + str(goal_events['Event period minute'][index]) + ". " for index, team in enumerate(goal_events['Team'])]

print(event_prompt)

['Liverpool scoorde in minuut 39. ', 'Liverpool scoorde in minuut 52. ', 'Liverpool scoorde in minuut 84. ', 'Liverpool scoorde in minuut 25. ']


# Get Home/Away team

In [10]:
get_home_team = df.query('TEAM_ID == HOME_TEAM')
get_away_team = df.query('TEAM_ID != HOME_TEAM')

home_team = get_home_team.TEAM_NAME.value_counts().index.tolist()
away_team = get_away_team.TEAM_NAME.value_counts().index.tolist()

print(home_team[0])
print(away_team[0])

Liverpool
Brighton and Hove Albion


# Get score

In [11]:
home_team_score = get_home_team.GOALS.value_counts().index.tolist()
away_team_score = get_away_team.GOALS.value_counts().index.tolist()

score_dict = {
    'Teams': [home_team[0], away_team[0]],
    'Goals': [home_team_score[0], away_team_score[0]]
}
print(score_dict)

{'Teams': ['Liverpool', 'Brighton and Hove Albion'], 'Goals': [4, 0]}


# Get winning team

In [12]:
winning_team_df = df.sort_values(by="GOALS", ascending=False)
winning_team = winning_team_df.TEAM_NAME.iloc[0]
print(winning_team)

losing_team_df = df.sort_values(by="GOALS", ascending=True)
losing_team = losing_team_df.TEAM_NAME.iloc[0]
print(losing_team)

Liverpool
Brighton and Hove Albion


# Possession Percentage

In [13]:
home_team_possession = get_home_team.POSSESSION_PERCENTAGE.value_counts().index.tolist()
away_team_possession = get_away_team.POSSESSION_PERCENTAGE.value_counts().index.tolist()

print(home_team_possession[0], home_team[0])
print(away_team_possession[0], away_team[0])

72.3 Liverpool
27.7 Brighton and Hove Albion


In [14]:
prompt = """Schrijf een wedstrijd samenvatting met minimaal 100 woorden en met maximaal 200 woorden over de tekst.
            Schrijf in de stijl van een sport verslaggever.
            tekst: 
                Wedstrijd: """+str(home_team[0])+""" speelt thuis tegen """+str(away_team[0])+""".
                Events: """+"".join(event_prompt)+"""
                Balbezit: """+str(home_team[0])+""" Had """+str(home_team_possession[0])+""" procent balbezit en """+str(away_team[0])+""" had """+str(away_team_possession[0])+""" procent balbezit.
                Eindstand: """+str(winning_team)+""" heeft gewonnen met """+str(score_dict['Goals'][0])+"""-"""+str(score_dict['Goals'][1])+"""."""

In [15]:
print(prompt)

Schrijf een wedstrijd samenvatting met minimaal 100 woorden en met maximaal 200 woorden over de tekst.
            Schrijf in de stijl van een sport verslaggever.
            tekst: 
                Wedstrijd: Liverpool speelt thuis tegen Brighton and Hove Albion.
                Events: Liverpool scoorde in minuut 39. Liverpool scoorde in minuut 52. Liverpool scoorde in minuut 84. Liverpool scoorde in minuut 25. 
                Balbezit: Liverpool Had 72.3 procent balbezit en Brighton and Hove Albion had 27.7 procent balbezit.
                Eindstand: Liverpool heeft gewonnen met 4-0.


# Create Dataframe: Prompt + Completion

In [16]:
prompt_and_completion = {
    'Prompt': str(prompt), 
    'Completion': "Hier komt de gehele samenvatting die bij de wedstrijd hoort."
}

In [17]:
pd.set_option("max_colwidth", None)
prompt_completion_df = pd.DataFrame(prompt_and_completion, index=[0])
display(prompt_completion_df)


,Prompt,Completion
0,Schrijf een wedstrijd samenvatting met minimaal 100 woorden en met maximaal 200 woorden over de tekst.\n Schrijf in de stijl van een sport verslaggever.\n tekst: \n Wedstrijd: Liverpool speelt thuis tegen Brighton and Hove Albion.\n Events: Liverpool scoorde in minuut 39. Liverpool scoorde in minuut 52. Liverpool scoorde in minuut 84. Liverpool scoorde in minuut 25. \n Balbezit: Liverpool Had 72.3 procent balbezit en Brighton and Hove Albion had 27.7 procent balbezit.\n Eindstand: Liverpool heeft gewonnen met 4-0.,Hier komt de gehele samenvatting die bij de wedstrijd hoort.


In [18]:
df_converted = pd.read_csv("./converted_prompts/opta_converted.csv", index_col=0)
display(df_converted)

,Prompt,Completion
0,Schrijf een wedstrijd samenvatting met minimaal 100 woorden en met maximaal 200 woorden over de tekst.\n Schrijf in de stijl van een sport verslaggever.\n tekst: \n Wedstrijd: Liverpool speelt thuis tegen Brighton and Hove Albion.\n Events: Liverpool scoorde in minuut 39. Liverpool scoorde in minuut 52. Liverpool scoorde in minuut 84. Liverpool scoorde in minuut 25. \n Balbezit: Liverpool Had 72.3 procent balbezit en Brighton and Hove Albion had 27.7 procent balbezit.\n Eindstand: Liverpool heeft gewonnen met 4-0.,Hier komt de gehele samenvatting die bij de wedstrijd hoort.


In [5]:
import datetime
import pytz
import calendar
import locale

locale.setlocale(category=locale.LC_ALL, locale='nl_NL')

created_at = datetime.datetime.now(pytz.timezone("Europe/Amsterdam"))
created_at_formatted = (
    str(calendar.month_name[created_at.month])
    + " "
    + str(created_at.day)
    + ": "
    + str(created_at.strftime("%H:%M:%S"))
)

In [6]:
created_at_formatted

'maart 23: 10:53:48'